In [1]:
import consMDP
import json
from decimal import Decimal
from tools import CMDPgraph

In [2]:
file_path = 'NYCstreetnetwork.json'
with open(file_path,'r') as f:
        graph_file = json.load(f)

In [3]:
%%timeit
chase = CMDPgraph.graph_from_json(file_path)[0]

38.8 s ± 3.81 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
chase = CMDPgraph.graph_from_json(file_path)[0]
type(chase)

tools.CMDPgraph.CMDP

In [5]:
def convert(g):
    states = []
    actions = dict()

    mdp = consMDP.ConsMDP()

    for node in g["nodes"]:
        if node["action"]:
            actions[node["label"]] = dict()
        else:
            states.append(node)

    for s in states:
        mdp.new_state(s["reload"], s["label"])

    for edge in g["edges"]:
        fr = edge["tail"]
        to = edge["head"]
        if to in actions:
            actions[to]["from"] = fr
            actions[to]["cons"] = edge["consumption"]
        else:
            dist = actions[fr].get("dist")
            to = mdp.state_with_label(to)
            if dist is None:
                actions[fr]["dist"] = dict()
            actions[fr]["dist"][to] = Decimal(f'{edge["probability"]}')

    for label, a in actions.items():
        fr = mdp.state_with_label(a["from"])
        mdp.add_action(fr, a["dist"], label, a["cons"])

    return mdp

In [6]:
%timeit m = convert(graph_file)

84.3 ms ± 1.04 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
m = convert(graph_file)
type(m)

consMDP.ConsMDP

In [10]:
%%timeit
m.compute_minInitCons();

589 ms ± 3.98 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
m.compute_minInitCons(10);

140 ms ± 1.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
